### Cine21 영화배우 정보 웹스크래핑
1. Site 주소 : http://www.cine21.com/rank/person/
2. Request URL : http://www.cine21.com/rank/person/content   
  - 크롬 개발자 도구의 network 탭에서 xhr 인 것만 필터링 
3. Request Method : POST
4. Form Data 
 - section: actor
 - period_start: 2020-09
 - gender: all
 - page: 1
5. 2년치 데이터를 한꺼번에 스크래핑 해서 actors.json 파일로 저장 

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin


### 1단계 
* 1 page 7명의 이름과 상세정보 추출

### 정규표현식 (regular expression)
* \w는 word를 표현하며 알파벳 + 숫자 + _ 중의 한 문자임을 의미한다.
* 문자가 출현한 횟수를 나타낼때 
 <pre>   
  * : 0 ~ n,  + : 1 ~ n , ? : 0(zero) or 1 
 </pre> 
* 정규표현식에 특수문자를 사용할때는 특수문자 앞에 \(back slash)를 반드시 주어야 한다
* .(period)는 줄바꿈 문자(\n)을 제외한 모든 글자 1개를 의미한다
* Greedy vs Non-Greedy
 - Greedy 욕심많은 (.*) : 모든 태그가 다 선택
 - Non-Greedy 욕심없는 (.*?) : 첫번째 매칭되는 태그만 선택
* [\n\r\t] : \n,\r,\t 중에 하나만 찾는다.
 - re.compile('[\n\r\t]')는 해당 정규표현식을 찾아주는 컴파일러를 생성한다.
 - compile 함수내의 정규표현식 앞에는 항상 r 을 붙혀 주어야 한다.

In [2]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all',
    'page': 1
}
res = requests.post(req_url, data=req_param)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    
    for li_tag in soup.select('li.people_li'):
        #print(li_tag.select('div.name'))
        div_tag = li_tag.select_one('div.name')
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        name = div_tag.text
        result = re.sub('\(\w*\)','',name)
        #print(result, detail_url)
                       
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                #<li>배우</li> => 배우
                actor_dict_value = re.sub('<.*?>','',result)
                
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub('',actor_dict_value)        
                print(actor_dict_key,actor_dict_value)
                #print(li_tag)

200
직업 배우
생년월일 1967-04-27
성별 남
신장/체중 177cm, 78kg
학교 유한대학 기계설계학
직업 배우
생년월일 1978-06-28
성별 여
홈페이지 https://twitter.com/hajiwon1023https://instagram.com/hajiwon1023
신장/체중 168cm, 45kg
학교 단국대학교 연극영화학 학사
취미 만화책보기, 소품만들기, 청소하기
특기 검도, 골프, 승마, 재즈댄스, 장구춤
직업 배우
생년월일 1971-01-10
성별 남
신장/체중 180cm
학교 서울예술대학 연극과
소속사 DMS커뮤니케이션즈
다른 이름 엄홍식
직업 배우
생년월일 1986-10-06
성별 남
홈페이지 https://www.instagram.com/hongsick/https://twitter.com/seeksik
신장/체중 178cm, 60kg
학교 단국대학교 연극
취미 피아노 연주, 인터넷 게임, 영화, 음악감상
특기 스노우보드, 수영
직업 배우
생년월일 2012-03-12
성별 여
직업 배우
생년월일 1973-06-03
성별 남
직업 배우
생년월일 1941-11-30
성별 여
홈페이지 http://www.namuni.kr/
학교 창덕여자고등학교


### 2단계 
* 1 page 7명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.

In [3]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all',
    'page': 1
}
res = requests.post(req_url, data=req_param)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    
    actor_detail_list = []
    actor_li_tag = soup.select('li.people_li')    
    
    for li_tag in actor_li_tag:
        actor_detail_dict = {}
        
        #print(li_tag.select('div.name'))
        div_tag = li_tag.select_one('div.name')
        #상세페이지 url 추출
        detail_url = div_tag.select_one('a')['href']
        detail_url = urljoin(req_url,detail_url)
        
        #배우이름 추출
        name = div_tag.text
        #성동일(1편) => 성동일 
        actor_name = re.sub('\(\w*\)','',name)
        actor_detail_dict['배우이름'] = actor_name
        
        #순위(ranking) 추출
        #print(li_tag.select_one('span.grade'))                   
        actor_ranking = li_tag.select_one('span.grade').text
        actor_detail_dict['랭킹'] = int(actor_ranking)
        
        #흥행지수 추출
        actor_hit = li_tag.select_one('ul.num_info > li > strong').text
        actor_detail_dict['흥행지수'] = int(actor_hit.replace(',',''))
        
        #상세정보 페이지에 요청을 보냄    
        res = requests.get(detail_url)
        if res.ok:
            soup = BeautifulSoup(res.text,'html.parser')
            
            #기본정보
            for li_tag in soup.select('ul.default_info li'):
                actor_dict_key = li_tag.select_one('span.tit').text
                #print(li_tag)
                #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                #<li>배우</li> => 배우
                actor_dict_value = re.sub('<.*?>','',result)
                
                regex = re.compile(r'[\n\r\t]')
                actor_dict_value = regex.sub('',actor_dict_value)        
                #print(actor_dict_key,actor_dict_value)
                
                #상세페이지에 추출한 key와 value를 dict에 저장
                actor_detail_dict[actor_dict_key] = actor_dict_value
            
            #참여작품
            movie_title_list = []
            for movie_li_tag in soup.select('ul.part_works li'):
                movie_title = movie_li_tag.select_one('span.tit').text
                movie_title_list.append(movie_title)
            
            #print(movie_title_list)
            actor_detail_dict['출연영화'] = movie_title_list
            
            #상세정보 담은 dict를 list에 추가
            actor_detail_list.append(actor_detail_dict)

print(actor_detail_list[0:2])            

200
[{'배우이름': '성동일', '랭킹': 1, '흥행지수': 137320, '직업': '배우', '생년월일': '1967-04-27', '성별': '남', '신장/체중': '177cm, 78kg', '학교': '유한대학 기계설계학', '출연영화': ['담보', '방법', '어쩌다 가족']}, {'배우이름': '하지원', '랭킹': 2, '흥행지수': 102990, '직업': '배우', '생년월일': '1978-06-28', '성별': '여', '홈페이지': 'https://twitter.com/hajiwon1023https://instagram.com/hajiwon1023', '신장/체중': '168cm, 45kg', '학교': '단국대학교 연극영화학 학사', '취미': '만화책보기, 소품만들기, 청소하기', '특기': '검도, 골프, 승마, 재즈댄스, 장구춤', '출연영화': ['담보', '초콜릿', '갈릴레오 : 깨어난 우주']}]


### 3단계 
* 1,2,3 page 21명의 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.

In [4]:
req_url = 'http://www.cine21.com/rank/person/content'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all'
}

#21명의 배우정보 dict를 저장할 리스트
actor_detail_list = []

for page in range(1,4):
    req_param['page'] = page
    res = requests.post(req_url, data=req_param)
    #print(res.status_code)
    print(f'---> {page} 페이지 출력됨')
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        actor_li_tag = soup.select('li.people_li')
        for idx,li_tag in enumerate(actor_li_tag,1):
            actor_detail_dict = {}

            #print(li_tag.select('div.name'))
            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #배우이름 추출
            name = div_tag.text
            #성동일(1편) => 성동일 
            actor_name = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = actor_name
            
            print(f'======> {idx} : {actor_name}')

            #순위(ranking) 추출
            #print(li_tag.select_one('span.grade'))                   
            actor_ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(actor_ranking)

            #흥행지수 추출
            actor_hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = actor_hit.replace(',','')

            #상세정보 페이지에 요청을 보냄    
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',result)

                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub('',actor_dict_value)        
                    #print(actor_dict_key,actor_dict_value)

                    #상세페이지에 추출한 key와 value를 dict에 저장
                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #참여작품
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)

                #print(movie_title_list)
                actor_detail_dict['출연영화'] = movie_title_list

                #상세정보 담은 dict를 list에 추가
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))                

---> 1 페이지 출력됨
======> 1 : 성동일
======> 2 : 하지원
======> 3 : 김희원
======> 4 : 유아인
======> 5 : 박소이
======> 6 : 유재명
======> 7 : 나문희
---> 2 페이지 출력됨
======> 1 : 이희준
======> 2 : 김호중
======> 3 : 고아성
======> 4 : 이솜
======> 5 : 박혜수
======> 6 : 최원영
======> 7 : 조현철
---> 3 페이지 출력됨
======> 1 : 곽도원
======> 2 : 김대명
======> 3 : 문승아
======> 4 : 김종수
======> 5 : 김원해
======> 6 : 김상호
======> 7 : 신민아
21


### 4단계 
* 모든 page의 배우 이름,흥행지수,랭킹과 상세정보 추출
* 배우의 정보는 dict에 저장하고, dict를 list에 저장한다.
* itertools 모듈의 count() 함수를 사용하여 무한 Looping
 - 무한 Loop를 탈출할 수 있는 조건
 - actor_li_tag = soup.select('li.people_li') 의 actor_list_tag의 길이가 0 이면 탈출

In [5]:
from itertools import count

req_url = 'http://www.cine21.com/rank/person/content'
#2년치 데이터는 'period_start': '2018-10'
req_param = {
    'section': 'actor',
    'period_start': '2020-09',
    'gender': 'all'
}

#21명의 배우정보 dict를 저장할 리스트
actor_detail_list = []

for page in count(1):
    req_param['page'] = page
    res = requests.post(req_url, data=req_param)
    #print(res.status_code)
    if page % 50 == 0:
        print(f'---> {page} 페이지 출력됨')
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')

        actor_li_tag = soup.select('li.people_li')
        
        #해당 page가 없으면 loop를 탈출
        if len(actor_li_tag) == 0:
            break
            
        for idx,li_tag in enumerate(actor_li_tag,1):
            actor_detail_dict = {}

            #print(li_tag.select('div.name'))
            div_tag = li_tag.select_one('div.name')
            #상세페이지 url 추출
            detail_url = div_tag.select_one('a')['href']
            detail_url = urljoin(req_url,detail_url)

            #배우이름 추출
            name = div_tag.text
            #성동일(1편) => 성동일 
            actor_name = re.sub('\(\w*\)','',name)
            actor_detail_dict['배우이름'] = actor_name
            
            print(f'======> {idx} : {actor_name}')

            #순위(ranking) 추출
            #print(li_tag.select_one('span.grade'))                   
            actor_ranking = li_tag.select_one('span.grade').text
            actor_detail_dict['랭킹'] = int(actor_ranking)

            #흥행지수 추출
            actor_hit = li_tag.select_one('ul.num_info > li > strong').text
            actor_detail_dict['흥행지수'] = int(actor_hit.replace(',',''))

            #상세정보 페이지에 요청을 보냄    
            res = requests.get(detail_url)
            if res.ok:
                soup = BeautifulSoup(res.text,'html.parser')

                #기본정보
                for li_tag in soup.select('ul.default_info li'):
                    actor_dict_key = li_tag.select_one('span.tit').text
                    #print(li_tag)
                    #<li><span class="tit">직업</span>배우</li> => <li>배우</li>
                    result = re.sub('<span.*>.*<\/span>','',str(li_tag))
                    #<li>배우</li> => 배우
                    actor_dict_value = re.sub('<.*?>','',result)

                    regex = re.compile(r'[\n\r\t]')
                    actor_dict_value = regex.sub('',actor_dict_value)        
                    #print(actor_dict_key,actor_dict_value)

                    #상세페이지에 추출한 key와 value를 dict에 저장
                    actor_detail_dict[actor_dict_key] = actor_dict_value

                #참여작품
                movie_title_list = []
                for movie_li_tag in soup.select('ul.part_works li'):
                    movie_title = movie_li_tag.select_one('span.tit').text
                    movie_title_list.append(movie_title)

                #print(movie_title_list)
                actor_detail_dict['출연영화'] = movie_title_list

                #상세정보 담은 dict를 list에 추가
                actor_detail_list.append(actor_detail_dict)

print(len(actor_detail_list))                        

======> 1 : 성동일
======> 2 : 하지원
======> 3 : 김희원
======> 4 : 유아인
======> 5 : 박소이
======> 6 : 유재명
======> 7 : 나문희
======> 1 : 이희준
======> 2 : 김호중
======> 3 : 고아성
======> 4 : 이솜
======> 5 : 박혜수
======> 6 : 최원영
======> 7 : 조현철
======> 1 : 곽도원
======> 2 : 김대명
======> 3 : 문승아
======> 4 : 김종수
======> 5 : 김원해
======> 6 : 김상호
======> 7 : 신민아
======> 1 : 장혁
======> 2 : 배해선
======> 3 : 황정민
======> 4 : 박지영
======> 5 : 이선호
======> 6 : 유진우
======> 7 : 김현수
======> 1 : 백현진
======> 2 : 손현주
======> 3 : 이유영
======> 4 : 이성욱
======> 5 : 조재윤
======> 6 : 정만식
======> 7 : 임영웅
======> 1 : 이정재
======> 2 : RM
======> 3 : 신동미
======> 4 : 이정현
======> 5 : 이주영
======> 6 : 영탁
======> 7 : 진
======> 1 : 이나경
======> 2 : 신승환
======> 3 : 이진주
======> 4 : 홍시호
======> 5 : 슈가
======> 6 : 김성오
======> 7 : 이찬원
======> 1 : 이한서
======> 2 : 제이홉
======> 3 : 이규형
======> 4 : 이민혁
======> 5 : 문남숙
======> 6 : 서영희
======> 7 : 정동원
======> 1 : 박정민
======> 2 : 장민혁
======> 3 : 김창옥
======> 4 : 조현정
======> 5 : 지민
======> 6 : 김보영
======> 7 : 양동근


======> 5 : 안재홍
======> 6 : 이효제
======> 7 : 전영선
======> 1 : 진지희
======> 2 : 권율
======> 3 : 민영
======> 4 : 나정옥
======> 5 : 한효주
======> 6 : 이양희
======> 7 : 이정애
======> 1 : 이정애
======> 2 : 서예지
======> 3 : 백수장
======> 4 : 곽진무
======> 5 : 고서희
======> 6 : 김도영
======> 7 : 김새론
======> 1 : 최준영
======> 2 : 서정
======> 3 : 남연우
======> 4 : 최재호
======> 5 : 김신재
======> 6 : 김종구
======> 7 : 엄지성
======> 1 : 한신화
======> 2 : 배소은
======> 3 : 공형진
======> 4 : 박세준
======> 5 : 김준한
======> 6 : 김은하
======> 7 : 윤혜리
======> 1 : 남연우
======> 2 : 최재호
======> 3 : 김신재
======> 4 : 김종구
======> 5 : 엄지성
======> 6 : 한신화
======> 7 : 배소은
======> 1 : 김새론
======> 2 : 조혜주
======> 3 : 김도영
======> 4 : 이현
======> 5 : 김지영
======> 6 : 김자영
======> 7 : 김경익
======> 1 : 전소니
======> 2 : 김주희
======> 3 : 송명진
======> 4 : 송요셉
======> 5 : 김동석
======> 6 : 이종원
======> 7 : 나애심
======> 1 : 전소니
======> 2 : 김주희
======> 3 : 송명진
======> 4 : 송요셉
======> 5 : 김동석
======> 6 : 이종원
======> 7 : 나애심
======> 1 : 이대연
======> 2 : 배제기
======> 3 : 김주아
======> 4 : 

In [6]:
for actor in actor_detail_list:
    if len(actor['배우이름']) > 10:
        print(actor)

In [7]:
print('before', len(actor_detail_list))
actor_detail_final_list = [actor for actor in actor_detail_list \
                           if len(actor['배우이름']) < 10]
print('after', len(actor_detail_final_list))

before 600
after 600


### 5단계
* actor_detail_final_list를 actors.json 파일로 저장하기

In [8]:
import json

with open('data/actors2.json','w',encoding='utf8') as file:
    json.dump(actor_detail_final_list, file)

print('json file write 종료')

json file write 종료


In [9]:
with open('data/actors2.json','r') as file:
    json_data = json.load(file)

print(len(json_data))    

600
